1) Tout d'abord, importez les packages nécessaires dans votre code. Cela inclut nltk, streamlit et speech_recognition.

In [27]:
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import speech_recognition as sr
import nltk
import io
import string

In [4]:
nltk.download('punkt', quiet=True)

True

2) Chargez le fichier texte et prétraitez les données à l’aide de l’algorithme du chatbot.

In [14]:
# chargement du fichier text
chemin_fichier = r"C:\Users\DELL\Desktop\reconnaissance_vocal\chatbot+reconnaissance\statistique.txt"
try:
    with open(chemin_fichier, 'r', encoding='utf-8') as fichier:
        contenu = fichier.read()
        #print(contenu)
except FileNotFoundError:
    print("Le fichier n'a pas été trouvé. Vérifiez le chemin et l'existence du fichier.")
except Exception as e:
    print(f"Une erreur est survenue : {e}")

In [18]:
# Dictionnaire de questions avec plusieurs réponses possibles
responses = {
    "qu'est-ce que la fonction de répartition ?": [
        "Soit X une variable aléatoire, on appelle fonction de répartition de X, que l’on note F_X, "
        "la fonction définie sur R par : F_X(x) = P(X ≤ x)."
    ],
    "qu'est-ce qu'une quantile d'ordre q ?": [
        "On appelle quantile d’ordre q de la variable X, où q ∈ [0, 1], la valeur xq telle que P(X ≤ xq) = q "
        "ou, de même, FX(xq) = q."
    ],
    "qu'est-ce que la théorie des probabilités ?": [
        """La théorie des probabilités a pour objet l’étude des phénomènes aléatoires<br>
        ou du moins considérés comme tels par l’observateur. Pour cela, on introduit le concept d’expérience <br>
        aléatoire dont l’ensemble des résultats possibles constitue l’ensemble fondamental, noté habituellement Ω."""
    ],
    "qu'est-ce qu'une variable aléatoire ?": [
        """On parle de variable aléatoire (abréviation : v.a.) lorsque les résultats sont numériques, c’est-à-dire<br>
        que Ω est identique à tout ou une partie de l’ensemble des nombres réels R.<br>
        On distingue habituellement :<br>
        - les variables aléatoires discrètes pour lesquelles l’ensemble Ω des résultats possibles est un <br>
        ensemble discret de valeurs numériques x1, x2, ..., xn,<br>
        fini ou infini (typiquement : l’ensemble des entiers naturels) ;<br>
        - les variables aléatoires continues pour lesquelles l’ensemble Ω est tout R<br>
        (ou un intervalle de R ou, plus rarement, une union d’intervalles)."""
    ],
    "qu'est-ce qu'une espérance mathématique ?": [
        """On appelle espérance mathématique de X, si elle existe, la valeur notée E(X) telle que :<br>
        E(X) = Σ (xi * pX(xi)) dans le cas discret,<br>
        E(X) = ∫ (x * fX(x) dx) dans le cas continu.<br>
        Du point de vue du graphe de fX (respectivement pX), cette valeur représente le <br>
        centre de gravité de la distribution."""
    ],
    "qu'est-ce que l'espérance d'une fonction ?": [
        """Soit g(X) une fonction de la v.a. X, alors :<br>
        E(g(X)) = ∫ g(x) fX(x) dx dans le cas continu (si l’intégrale existe),<br>
        E(g(X)) = Σ g(xi) pX(xi) dans le cas discret (si la somme existe).<br>
        On voit donc que pour le calcul de E(g(X)), il suffit de remplacer la valeur de x par g(x) (ou xi par g(xi))."""
    ]
}

In [20]:
# Prétraitement du texte
def preprocess_text(text):
    """Prétraiter le texte : conversion en minuscules et suppression de la ponctuation."""
    text = text.lower()  # Convertir en minuscules
    text = text.translate(str.maketrans('', '', string.punctuation))  # Supprimer la ponctuation
    tokens = nltk.word_tokenize(text)  # Tokenisation
    return ' '.join(tokens)  # Retourner le texte prétraité sous forme de chaîne

3) Définir une fonction pour transcrire la parole en texte à l'aide de l'algorithme de reconnaissance vocale.

In [24]:
def transcribe_speech(api_choice, language, listening):
    r = sr.Recognizer()
    
    if not listening:
        return "L'écoute est mise en pause."

    with sr.Microphone() as source:
        st.info("Parlez maintenant...")
        audio_text = r.listen(source)
        st.info("Transcription...")

        try:
            # Sélection de l'API de reconnaissance vocale
            if api_choice == "Google":
                text = r.recognize_google(audio_text, language=language)
            elif api_choice == "Sphinx":
                text = r.recognize_sphinx(audio_text)
            else:
                return "Erreur : API non reconnue."

            return text

        except sr.UnknownValueError:
            return "Erreur : Impossible de comprendre l'audio."
        except sr.RequestError as e:
            return f"Erreur de service : {e}"
        except Exception as e:
            return f"Une erreur inattendue est survenue : {e}"

4) Modifiez la fonction du chatbot pour qu'il reçoive à la fois des entrées textuelles et vocales de l'utilisateur. Si l'utilisateur fournit une entrée textuelle, le chatbot doit fonctionner comme avant. Si l'utilisateur fournit une entrée vocale, l'algorithme de reconnaissance vocale doit transcrire la parole en texte, qui est ensuite transmis au chatbot.

In [29]:
def normalize_query(query):
    """Normaliser la requête de l'utilisateur."""
    return query.lower().strip()

In [31]:
def get_most_relevant_sentence(user_query, text_data):
    """Trouver la phrase la plus pertinente à partir du texte prétraité."""
    sentences = ' '.join(text_data).split('. ')
    sentences.append(user_query)
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(sentences)
    cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])
    most_relevant_index = cosine_similarities.argmax()
    return sentences[most_relevant_index]

In [33]:
def chatbot(user_query, text_data):
    """Fonction principale du chatbot."""
    normalized_query = normalize_query(user_query)

    # Vérification des réponses directes
    for question in responses:
        if question in normalized_query:
            return responses[question][0]
    
    # Si aucune réponse directe, trouver la phrase la plus pertinente
    return get_most_relevant_sentence(user_query, text_data)

In [35]:
def main():
    st.title("Chatbot Statistique et Probabilités")
    
    # Sélecteur d'API de reconnaissance vocale
    api_choice = st.selectbox("Choisissez l'API de reconnaissance vocale :", ["Google", "Sphinx"])
    
    # Sélecteur de langue
    language = st.selectbox("Choisissez la langue :", ["fr-FR", "en-US", "es-ES", "de-DE"])
    
    # Chargement du fichier texte
    file_path = st.text_input("Entrez le chemin du fichier texte :", "statistique.text")
    
    if st.button("Charger et Prétraiter le Fichier"):
        try:
            raw_text = load_text_file(file_path)
            text_data = preprocess_text(raw_text)
            st.write("Texte prétraité : ", text_data)
        except FileNotFoundError:
            st.write("Le fichier n'a pas été trouvé. Veuillez vérifier le chemin.")

    # Zone de saisie pour poser une question
    user_input = st.text_input("Posez votre question sur la statistique et la probabilité :")
    
    if user_input and 'text_data' in locals():
        response = chatbot(user_input, text_data)
        st.markdown(response, unsafe_allow_html=True)

    # Option pour la saisie vocale
    if st.button("Transcrire la parole"):
        speech_text = transcribe_speech(api_choice, language)
        if 'text_data' in locals():
            response = chatbot(speech_text, text_data)
            st.markdown(response, unsafe_allow_html=True)

if __name__ == "__main__":
    main()

2024-12-17 13:49:41.328 
  command:

    streamlit run C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-12-17 13:49:41.328 Session state does not function when running a script without `streamlit run`


5) Créez une application Streamlit qui permet à l'utilisateur de fournir une entrée de texte ou de parole au chatbot. Si l'utilisateur fournit une entrée de texte, le chatbot doit fonctionner comme avant. Si l'utilisateur fournit une entrée vocale, l'algorithme de reconnaissance vocale doit transcrire la parole en texte, qui est ensuite transmis au chatbot. La réponse du chatbot doit être affichée à l'utilisateur.